In [5]:
import time
import sys
sys.path.append("c:\\users\\valen\\GitHub\\Lib")
print(sys.path)
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.opinions as op
import future.utils

['C:\\Users\\valen\\Github\\AlgorithmicBias', 'c:\\users\\valen\\github\\lib', 'c:\\programdata\\anaconda3', 'c:\\programdata\\anaconda3\\lib', '', 'c:\\programdata\\anaconda3\\lib\\site-packages', 'C:\\Users\\valen\\Github\\AlgorithmicBias', 'C:\\ProgramData\\Anaconda3\\python37.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\Users\\valen\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\cdlib-0.1.10-py3.7.egg', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\pooch-1.3.0-py3.7.egg', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\appdirs-1.4.4-py3.7.egg', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'c:\\programdata\\anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\valen\\.ipython', 'c:\\users\\valen\\GitHub\\Lib', 'c:\\users\\valen\\GitHub\\Lib']


# Spaghetti plot

In [6]:
class OpinionEvolution(object):
    
    def __init__(self, model, trends):
        """
        :param model: The model object
        :param trends: The computed simulation trends
        """
        self.model = model
        self.srev = trends
        self.ylabel = "Opinion"
    
    def plot(self, fig, ax, filename=None):
        nodes2opinions = {}
        node2col = {}

        last_it = self.srev[-1]['iteration'] + 1
        last_seen = {}

        for it in self.srev:
            sts = it['status']
            its = it['iteration']
            for n, v in sts.items():
                if n in nodes2opinions:
                    last_id = last_seen[n]
                    last_value = nodes2opinions[n][last_id]

                    for i in range(last_id, its):
                        nodes2opinions[n][i] = last_value

                    nodes2opinions[n][its] = v
                    last_seen[n] = its
                else:
                    nodes2opinions[n] = [0]*last_it
                    nodes2opinions[n][its] = v
                    last_seen[n] = 0
                    if v < 0.4:
                        node2col[n] = '#3776ab'
                    elif 0.4 <= v <= 0.6:
                        node2col[n] = '#FFA500'
                    else:
                        node2col[n] = '#FF0000'       
        mx = 0
        for k, l in future.utils.iteritems(nodes2opinions):
            if mx < last_seen[k]:
                mx = last_seen[k]
            x = list(range(0, last_seen[k]))
            y = l[0:last_seen[k]]
            ax.plot(x, y, lw=1, alpha=0.5, color=node2col[k])

        ax.set_ylim(-0.1, 1.1)
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
        ax.tick_params(axis='both', which='major', labelsize=3, pad=2)                
        plt.grid(axis = 'both', which='both')
        plt.tight_layout()

In [12]:
def restoiterations(filename):   
    iterations = list()
    try:
        infile = open(filename, "r")
        lines = infile.readlines()
        for i,line in enumerate(lines):
            opinions = list(line.split(','))
            itdict = {"iteration":i, "status":{k: float(v) for k, v in enumerate(opinions)},"node_count":{0:0},"status_delta":{0:0}}
            iterations.append(itdict)
    except FileNotFoundError:
        pass
    return iterations

def spaghetti(e,p,g,fig=None,ax=None):
    try:
        graph = nx.complete_graph(100)
        model = op.AlgorithmicBiasModel(graph) 
        config = mc.Configuration()
        # config.add_model_parameter("p", p)
        # config.add_model_parameter("k",1)
        config.add_model_parameter("epsilon", e)
        config.add_model_parameter("gamma", g)
        # config.add_model_parameter("gamma_media",g)
        model.set_initial_status(config)
        # model.set_media_opinions([0.0])
    except Exception as err:
        print(err)
    
    iterations = restoiterations("res/media mo[0.0] p{} e{} g{} gm{} mi1000000 nr1".format(p,e,g,g))
    if len(iterations)>0:
        viz = OpinionEvolution(model, iterations)
    else:
        print("file not found returning")
        return
    
    if fig==None:
        print("saving spaghetti plot")
        viz.plot("plots/evolution media mo[0.0] p{} e{} g{} gm{} mi1000000 nr{}".format(p,e,g,g,nr))
    else:
        viz.plot(fig, ax)

def spaghettigrid(p):
    fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(6, 3), dpi=600, sharey=True)
    row=0
    for e in [0.2, 0.3, 0.5]:
        col=0
        for g in [0.0, 0.5, 0.75, 1.0, 1.5]: 
            spaghetti(e,p,g,fig=fig,ax=axes[row,col])
            col+=1
        row+=1
    print("saving spaghetti grid plot")
    plt.savefig("plots/evolution_grid media mo[0.0] p{} e{} g{} gm{} mi1000000.png".format(p,e,g,g))

def final_distribution(e,p,g,fig=None,ax=None):
    x = [i for i in range(100)]
    nr=1
    iterations = restoiterations("res/media mo[0.0] p{} e{} g{} gm{} mi1000000 nr{}.png".format(p,e,g,g,nr))
    if len(iterations)>0:
        y = sorted(list(iterations[-1]['status']))
        plt.scatter(x, y)
        print("saving final dist plot")
        plt.savefig("plots/final_distribution media mo[0.0] p{} e{} g{} gm{} mi1000000 nr{}".format(p,e,g,g,nr))

In [13]:
for e in [0.1, 0.2, 0.3, 0.4, 0.5, 1.0]:
    for g in [0.0, 0.5, 0.75, 1.0, 1.5]: 
        for p in [0.1, 0.2, 0.3, 0.4, 0.5]:
            spaghetti(e,p,g,fig=None,ax=None)
            final_distribution(e,p,g,fig=None,ax=None)

file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning
file not found returning


In [ ]:
start = time.time()
for p in [0.1, 0.2, 0.3, 0.4, 0.5]:
    spaghettigrid(p=p)
print("process lasted {} seconds".format(time.time()-start))